<a href="https://colab.research.google.com/github/yonabell/Report/blob/main/Reporting_final_python_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os
import re

In [2]:
from google.colab import files
uploaded = files.upload()

Saving 2023_sample_data1.xlsx to 2023_sample_data1 (5).xlsx


In [3]:
file_path_previous_year = '2023_sample_data1.xlsx'

In [ ]:
# file_path_current_year = 'path_to_input_folder/2024_sample_data.xlsx'
# file_path_previous_year = 'path_to_input_folder/2023_sample_data.xlsx'
# output_folder_path = 'path_to_output_folder/'

In [4]:
# Defining a function for the following data cleaning steps
def clean_data(file_path):
    # Load the data with no predefined header
    cpt_df = pd.read_excel(file_path, engine='openpyxl', header=4)

    # Filtering rows and extracting only 'A' followed by nine digits
    filtered_cpt_df = cpt_df[cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False).notna()].copy()

    # Extracting the ID
    filtered_cpt_df['ID'] = filtered_cpt_df['Unnamed: 0'].str.extract(r'(A\d{9})', expand=False)

    # Reorder columns to place 'ID' next to 'Unnamed: 0'
    cols = filtered_cpt_df.columns.tolist()
    id_index = cols.index('ID')
    cols = cols[:1] + [cols[id_index]] + cols[1:id_index] + cols[id_index + 1:]
    filtered_cpt_df = filtered_cpt_df[cols]

    # Drop 'Unnamed: 0' column
    filtered_cpt_df = filtered_cpt_df.drop('Unnamed: 0', axis=1)

    # Drop specific columns (1 to 7)
    filtered_cpt_df = filtered_cpt_df.drop(filtered_cpt_df.columns[1:8], axis=1)

    # Select every 7th column starting from the 8th column (index 7)
    # Unnecessary column for the analysis with no data
    column_names = filtered_cpt_df.columns.tolist()
    columns_to_drop = column_names[7::7]

    # Drop the selected columns from the DataFrame
    filtered_cpt_df.drop(columns=columns_to_drop, inplace=True)

    return filtered_cpt_df

In [5]:
# Call the clean_data function
filtered_cpt_df1 = clean_data(file_path_previous_year) # file_path_current_year

In [6]:
# Display the filtered_cpt_df
filtered_cpt_df1

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,...,Profit Margin I.11,PM I % of Purchased Volume.11,Profit Margin I*.11,PM I* % of Purchased Volume.11,Purch. Volume.12,S-Revenues.12,Profit Margin I.12,PM I % of Purchased Volume.12,Profit Margin I*.12,PM I* % of Purchased Volume.12
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [7]:
filtered_cpt_df1.shape

(2, 73)

In [7]:
def replace_suffix_with_months(df):
    """
    This function replaces numerical suffixes in column names with corresponding month names
    and replaces dots with spaces.

    Args:
    df (pd.DataFrame): The input DataFrame whose column names need to be updated.

    Returns:
    pd.DataFrame: The DataFrame with updated column names.
    """
    # Mapping of number suffixes to month names
    month_map = {
        '.1': ' Jan',
        '.2': ' Feb',
        '.3': ' Mar',
        '.4': ' Apr',
        '.5': ' May',
        '.6': ' Jun',
        '.7': ' Jul',
        '.8': ' Aug',
        '.9': ' Sep',
        '.10': ' Oct',
        '.11': ' Nov',
        '.12': ' Dec'
    }

    # Replace dots with spaces and map suffixes at the end of column names to respective months
    new_column_names = []
    for column_name in df.columns:
        new_column_name = column_name.replace('.', ' ')  # Replace dots with spaces
        for suffix, month_name in month_map.items():
            # Ensure we replace only if the suffix appears at the end of the column name
            if new_column_name.endswith(suffix.replace('.', ' ')):
                new_column_name = new_column_name.replace(suffix.replace('.', ' '), month_name)
        new_column_names.append(new_column_name)

    # Assign the new column names to the dataframe
    df.columns = new_column_names

    return df

In [8]:
filtered_cpt_df1 = replace_suffix_with_months(filtered_cpt_df1)

In [10]:
filtered_cpt_df1

,ID,Purch Volume Jan,S-Revenues Jan,Profit Margin I Jan,PM I % of Purchased Volume Jan,Profit Margin I* Jan,PM I* % of Purchased Volume Jan,Purch Volume Feb,S-Revenues Feb,Profit Margin I Feb,...,Profit Margin I Nov,PM I % of Purchased Volume Nov,Profit Margin I* Nov,PM I* % of Purchased Volume Nov,Purch Volume Dec,S-Revenues Dec,Profit Margin I Dec,PM I % of Purchased Volume Dec,Profit Margin I* Dec,PM I* % of Purchased Volume Dec
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [9]:
print(filtered_cpt_df1.shape)

(2, 73)


In [10]:
def rename_columns(filtered_cpt_df):
    # Define the new column name mappings for replacements at the start of column names
    new_column_names = {
        'ID': 'BPID',
        'Purch Volume': 'PurchVolume',
        'S-Revenues': 'Revenue',
        'Profit Margin I*': 'PMIStar',  # Handle the specific case for PMIStar first
        'Profit Margin I': 'PMI',
        'PM I % of Purchased Volume': 'PMIRate',
        'PM I* % of Purchased Volume': 'PMIStarRate'
    }

    # Clean column names by collapsing multiple spaces into a single space
    filtered_cpt_df.columns = [re.sub(r'\s+', ' ', col) for col in filtered_cpt_df.columns]

    # Iterate through the columns and apply the renaming based on the starting part of the name
    updated_columns = []
    for column in filtered_cpt_df.columns:
        # Check if the column starts with any of the keys in new_column_names
        for old_name, new_name in new_column_names.items():
            if column.startswith(old_name):
                # Replace the old part of the column name with the new one and keep the suffix (e.g., "Jan", "Feb")
                new_column = column.replace(old_name, new_name, 1)
                updated_columns.append(new_column)
                break
        else:
            # If no match, keep the column name unchanged
            updated_columns.append(column)

    # Rename the columns in the dataframe
    filtered_cpt_df.columns = updated_columns

    return filtered_cpt_df

In [11]:
filtered_cpt_df1 = rename_columns(filtered_cpt_df1)

In [14]:
filtered_cpt_df1

,BPID,PurchVolume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,PurchVolume Feb,Revenue Feb,PMI Feb,...,PMI Nov,PMIRate Nov,PMIStar Nov,PMIStarRate Nov,PurchVolume Dec,Revenue Dec,PMI Dec,PMIRate Dec,PMIStar Dec,PMIStarRate Dec
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,34.0,9.8384,45.0,2.033,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,643.0,15.7322,542.0,2.519,543.0,725.0,643.0,15.7322,542.0,2.519


In [15]:
filtered_cpt_df1.columns

Index(['BPID', 'PurchVolume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'PurchVolume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'PurchVolume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'PurchVolume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'PurchVolume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'PurchVolume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'PurchVolume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'PurchVolume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'PurchVolume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'PurchVolume Oct', 'Revenue Oct',
       'PMI Oc

In [12]:
from google.colab import files
uploaded = files.upload()

Saving 2024_sample_data.xlsx to 2024_sample_data (5).xlsx


In [13]:
file_path_current_year = '2024_sample_data.xlsx'

In [14]:
# Call the clean_data function
filtered_cpt_df2 = clean_data(file_path_current_year)

In [19]:
filtered_cpt_df2

,ID,Purch. Volume.1,S-Revenues.1,Profit Margin I.1,PM I % of Purchased Volume.1,Profit Margin I*.1,PM I* % of Purchased Volume.1,Purch. Volume.2,S-Revenues.2,Profit Margin I.2,PM I % of Purchased Volume.2,Profit Margin I*.2,PM I* % of Purchased Volume.2,Purch. Volume.3,S-Revenues.3,Profit Margin I.3,PM I % of Purchased Volume.3,Profit Margin I*.3,PM I* % of Purchased Volume.3
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [15]:
# Assuming 'filtered_cpt_df2' is your DataFrame
filtered_cpt_df2 = replace_suffix_with_months(filtered_cpt_df2)

In [21]:
filtered_cpt_df2

,ID,Purch Volume Jan,S-Revenues Jan,Profit Margin I Jan,PM I % of Purchased Volume Jan,Profit Margin I* Jan,PM I* % of Purchased Volume Jan,Purch Volume Feb,S-Revenues Feb,Profit Margin I Feb,PM I % of Purchased Volume Feb,Profit Margin I* Feb,PM I* % of Purchased Volume Feb,Purch Volume Mar,S-Revenues Mar,Profit Margin I Mar,PM I % of Purchased Volume Mar,Profit Margin I* Mar,PM I* % of Purchased Volume Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [16]:
filtered_cpt_df2 = rename_columns(filtered_cpt_df2)

In [23]:
filtered_cpt_df2

,BPID,PurchVolume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,PurchVolume Feb,Revenue Feb,PMI Feb,PMIRate Feb,PMIStar Feb,PMIStarRate Feb,PurchVolume Mar,Revenue Mar,PMI Mar,PMIRate Mar,PMIStar Mar,PMIStarRate Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [17]:
# Assuming df1 is filtered_cpt_df1 and you want to append columns from filtered_cpt_df2 starting from the second column

combined_df = pd.concat([filtered_cpt_df1, filtered_cpt_df2.iloc[:, 1:]], axis=1)


In [22]:
combined_df

,BPID,PurchVolume Jan,Revenue Jan,PMI Jan,PMIRate Jan,PMIStar Jan,PMIStarRate Jan,PurchVolume Feb,Revenue Feb,PMI Feb,...,PMI Feb,PMIRate Feb,PMIStar Feb,PMIStarRate Feb,PurchVolume Mar,Revenue Mar,PMI Mar,PMIRate Mar,PMIStar Mar,PMIStarRate Mar
5,A987654321,678.0,87464.0,7656.0,44.68,454.0,0.09,4355.0,43.0,2.0,...,2.0,0.14,546.0,3.785323,65.0,34.0,34.0,9.8384,45.0,2.033
8,A123456789,8876.0,3566.0,765.0,71.44,4326.0,0.12,1345.0,33.0,32.0,...,32.0,0.20,65.0,5.724400,543.0,725.0,643.0,15.7322,542.0,2.519


In [23]:
filtered_cpt_df1.shape



(2, 73)

In [27]:
filtered_cpt_df2.shape

(2, 19)

In [24]:
combined_df.shape

(2, 91)

In [26]:
filtered_cpt_df1.columns

Index(['BPID', 'PurchVolume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'PurchVolume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'PurchVolume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'PurchVolume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'PurchVolume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'PurchVolume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'PurchVolume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'PurchVolume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'PurchVolume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'PurchVolume Oct', 'Revenue Oct',
       'PMI Oc

In [ ]:
filtered_cpt_df2.columns

Index(['BPID', 'Purch  Volume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'Purch  Volume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'Purch  Volume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar'],
      dtype='object')

In [18]:
combined_df.columns

Index(['BPID', 'PurchVolume Jan', 'Revenue Jan', 'PMI Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStarRate Jan', 'PurchVolume Feb', 'Revenue Feb',
       'PMI Feb', 'PMIRate Feb', 'PMIStar Feb', 'PMIStarRate Feb',
       'PurchVolume Mar', 'Revenue Mar', 'PMI Mar', 'PMIRate Mar',
       'PMIStar Mar', 'PMIStarRate Mar', 'PurchVolume Apr', 'Revenue Apr',
       'PMI Apr', 'PMIRate Apr', 'PMIStar Apr', 'PMIStarRate Apr',
       'PurchVolume May', 'Revenue May', 'PMI May', 'PMIRate May',
       'PMIStar May', 'PMIStarRate May', 'PurchVolume Jun', 'Revenue Jun',
       'PMI Jun', 'PMIRate Jun', 'PMIStar Jun', 'PMIStarRate Jun',
       'PurchVolume Jul', 'Revenue Jul', 'PMI Jul', 'PMIRate Jul',
       'PMIStar Jul', 'PMIStarRate Jul', 'PurchVolume Aug', 'Revenue Aug',
       'PMI Aug', 'PMIRate Aug', 'PMIStar Aug', 'PMIStarRate Aug',
       'PurchVolume Sep', 'Revenue Sep', 'PMI Sep', 'PMIRate Sep',
       'PMIStar Sep', 'PMIStarRate Sep', 'PurchVolume Oct', 'Revenue Oct',
       'PMI Oc

In [45]:
# Create a mapping of month-year to column indices
month_year_mapping = {}
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
years = [2023, 2024]

# Populate the mapping
for year in years:
    for i, month in enumerate(months):
        # Calculate the index: 1 (for BPID) + (i * 6) for the month columns + (year - 2023) * 72 for the year offset
        month_year_mapping[f"{month} {year}"] = 1 + (i * 6) + (year - 2023) * 72

print(month_year_mapping)

{'Jan 2023': 1, 'Feb 2023': 7, 'Mar 2023': 13, 'Apr 2023': 19, 'May 2023': 25, 'Jun 2023': 31, 'Jul 2023': 37, 'Aug 2023': 43, 'Sep 2023': 49, 'Oct 2023': 55, 'Nov 2023': 61, 'Dec 2023': 67, 'Jan 2024': 73, 'Feb 2024': 79, 'Mar 2024': 85, 'Apr 2024': 91, 'May 2024': 97, 'Jun 2024': 103, 'Jul 2024': 109, 'Aug 2024': 115, 'Sep 2024': 121, 'Oct 2024': 127, 'Nov 2024': 133, 'Dec 2024': 139}


In [47]:
def filter_dataframe(df, start_month, end_month):
    # Convert start and end month strings to month-year format
    # Change: Extract month as integer and map it to month name
    start_month_num = int(start_month.split('.')[0])
    start_month_name = months[start_month_num - 1] # Get month name from list
    start_month_str = f"{start_month_name} {start_month.split('.')[1]}"

    end_month_num = int(end_month.split('.')[0])
    end_month_name = months[end_month_num - 1]  # Get month name from list
    end_month_str = f"{end_month_name} {end_month.split('.')[1]}"

    # Find the indices of the start and end month in the mapping
    start_index = month_year_mapping[start_month_str]
    end_index = month_year_mapping[end_month_str] + 6  # Include all 6 columns for the end month

    # Select the columns based on the calculated indices
    filtered_df = df.iloc[:, [0] + list(range(start_index, end_index))]

    return filtered_df

In [49]:
filtered_df.columns

Index(['01.2023', '01.2023', '01.2023', '01.2023', '01.2023', '01.2023'], dtype='object', name=0)

In [41]:
def filter_dataframe(df, start_month, end_month):
    # Convert start and end month strings to month-year format
    start_month_str = f"{start_month.split('.')[0]} {start_month.split('.')[1]}"
    end_month_str = f"{end_month.split('.')[0]} {end_month.split('.')[1]}"

    # Find the indices of the start and end month in the mapping
    start_index = month_year_mapping[start_month_str]
    end_index = month_year_mapping[end_month_str] + 3  # Include the end month columns

    # Select the columns based on the calculated indices
    filtered_df = df.iloc[:, [0] + list(range(start_index, end_index))]

    return filtered_df

# Example usage
start_month = '11.2023'
end_month = '1.2024'
filtered_df = filter_dataframe(combined_df, start_month, end_month)

# Display the filtered DataFrame
print(filtered_df)

In [33]:
filtered_df.columns

Index(['BPID', 'PurchVolume Jan', 'PurchVolume Jan', 'Revenue Jan',
       'Revenue Jan', 'PMI Jan', 'PMI Jan', 'PMIRate Jan', 'PMIRate Jan',
       'PMIStar Jan', 'PMIStar Jan', 'PMIStarRate Jan', 'PMIStarRate Jan',
       'PurchVolume Feb', 'PurchVolume Feb', 'Revenue Feb', 'Revenue Feb',
       'PMI Feb', 'PMI Feb', 'PMIRate Feb', 'PMIRate Feb', 'PMIStar Feb',
       'PMIStar Feb', 'PMIStarRate Feb', 'PMIStarRate Feb', 'PurchVolume Nov',
       'Revenue Nov', 'PMI Nov', 'PMIRate Nov', 'PMIStar Nov',
       'PMIStarRate Nov', 'PurchVolume Dec', 'Revenue Dec', 'PMI Dec',
       'PMIRate Dec', 'PMIStar Dec', 'PMIStarRate Dec'],
      dtype='object')

In [43]:
filtered_df.columns

Index(['01.2023', '01.2023', '01.2023', '01.2023', '01.2023', '01.2023'], dtype='object', name=0)

In [35]:
# Filter for 2023 and 2024 separately
filtered_cpt_df1 = combined_df.loc[:, combined_df.columns.str.contains('2023|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec')]
filtered_cpt_df2 = combined_df.loc[:, combined_df.columns.str.contains('2024|Jan|Feb|Mar|Apr|May|Jun')]

# Assign month-year identifiers to filtered_cpt_df1
month_mapping_2023 = {
    'Jan': '01.2023', 'Feb': '02.2023', 'Mar': '03.2023', 'Apr': '04.2023',
    'May': '05.2023', 'Jun': '06.2023', 'Jul': '07.2023', 'Aug': '08.2023',
    'Sep': '09.2023', 'Oct': '10.2023', 'Nov': '11.2023', 'Dec': '12.2023'
}

# Rename columns based on month mapping for 2023
for month, identifier in month_mapping_2023.items():
    filtered_cpt_df1.columns = [col if not col.endswith(month) else f"{col} ({identifier})" for col in filtered_cpt_df1.columns]

# Automatically create month mapping for 2024 based on existing columns
available_months_2024 = []
for month in filtered_cpt_df2.columns.str.extract(r'(\w+)$')[0].unique():
    if month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']:
        available_months_2024.append(month)

# Create month mapping for available months in 2024
month_mapping_2024 = {
    month: f"{str(i).zfill(2)}.2024" for i, month in enumerate(available_months_2024, start=1)
}

# Rename columns based on month mapping for 2024
for month, identifier in month_mapping_2024.items():
    filtered_cpt_df2.columns = [col if not col.endswith(month) else f"{col} ({identifier})" for col in filtered_cpt_df2.columns]

# Combine the two DataFrames
final_df = pd.concat([filtered_cpt_df1, filtered_cpt_df2], axis=1)

# Display the final result
print(final_df)

   PurchVolume Jan (01.2023)  Revenue Jan (01.2023)  PMI Jan (01.2023)  \
5                      678.0                87464.0             7656.0   
8                     8876.0                 3566.0              765.0   

   PMIRate Jan (01.2023)  PMIStar Jan (01.2023)  PMIStarRate Jan (01.2023)  \
5                  44.68                  454.0                       0.09   
8                  71.44                 4326.0                       0.12   

   PurchVolume Feb (02.2023)  Revenue Feb (02.2023)  PMI Feb (02.2023)  \
5                     4355.0                   43.0                2.0   
8                     1345.0                   33.0               32.0   

   PMIRate Feb (02.2023)  ...  PMI Feb (02.2024)  PMIRate Feb (02.2024)  \
5                   0.14  ...                2.0                   0.14   
8                   0.20  ...               32.0                   0.20   

   PMIStar Feb (02.2024)  PMIStarRate Feb (02.2024)  \
5                  546.0              

In [27]:
def filter_by_month_year(combined_df, start_month_year, end_month_year):
    """
    Filters the combined_df based on the specified start and end month-year combinations.

    Args:
        combined_df: The DataFrame to filter.
        start_month_year: The starting month-year (e.g., "11.2023").
        end_month_year: The ending month-year (e.g., "1.2024").

    Returns:
        A new DataFrame containing only the columns within the specified month-year range.
    """

    # Split the input month-year strings into integers
    start_month, start_year = map(int, start_month_year.split('.'))
    end_month, end_year = map(int, end_month_year.split('.'))

    # Initialize a list to hold columns to keep
    columns_to_keep = []

    # Iterate through the months and years from the start to end
    current_year = start_year
    current_month = start_month

    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
        # Format the month-year string
        column_suffix = f" {current_month}" if current_month < 10 else f" {current_month}"

        # Generate a list of columns for the current month and year
        for prefix in ['PurchVolume', 'Revenue', 'PMIStar', 'PMI', 'PMIRate', 'PMIStarRate']:
            for col_name in combined_df.columns:
                if col_name.endswith(column_suffix) and col_name.startswith(str(current_year)) and col_name.startswith(prefix):
                    columns_to_keep.append(col_name)

        # Move to the next month
        current_month += 1
        if current_month > 12:
            current_month = 1
            current_year += 1

    # Return a new DataFrame with only the columns to keep
    return combined_df[columns_to_keep] if columns_to_keep else combined_df.iloc[0:0]  # Return an empty DataFrame if no columns matched


In [34]:
# Example usage:
start_month_year = "11.2023"
end_month_year = "1.2024"
filtered_df = filter_by_month_year(combined_df, start_month_year, end_month_year)
filtered_df

,BPID
5,A987654321
8,A123456789


In [31]:
# Example usage
start_month = '11.2023'
end_month = '1.2024'
filtered_data = filter_by_date_range(combined_df, month_year_map, start_month, end_month)
print(filtered_data)

NameError: name 'filter_by_date_range' is not defined

In [ ]:
# Saving current year aggregated data to output folder
# aggregated_df1.to_excel(output_folder_path + '2024_aggregated.xlsx', index=False)

In [ ]:
# Saving current year aggregated data to output folder
# aggregated_df1.to_excel(output_folder_path + '2023_aggregated.xlsx', index=False)

In [ ]:
# Saving the final combined sum to output folder
# aggregated_df_sum.to_excel(output_folder_path + 'cpt_annual_profitability.xlsx', index=False)

In [ ]:
# Get the current and previous year dynamically
current_year = datetime.now().year
previous_year = current_year - 1

# Define input folder and output folder paths
input_folder_path = 'path_to_your_input/'
output_folder_path = 'path_to_output_folder/'

# Assign file paths dynamically
current_year_file_path = os.path.join(input_folder_path, f'{current_year}_sample_data.xlsx')
previous_year_file_path = os.path.join(input_folder_path, f'{previous_year}_sample_data.xlsx')

# Saving aggregated data for the current year
aggregated_df1.to_excel(os.path.join(output_folder_path, f'{current_year}_aggregated.xlsx'), index=False)

# Saving aggregated data for the previous year
aggregated_df2.to_excel(os.path.join(output_folder_path, f'{previous_year}_aggregated.xlsx'), index=False)
